In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="KQEdkoBrZB3iNhBNZDyx")
project = rf.workspace("api5").project("segmentacao-de-nuvens")
version = project.version(1)
dataset = version.download("png-mask-semantic")

In [ ]:
import os
import shutil

# Caminho da pasta onde estão as imagens e máscaras misturadas
source_dir = 'Segmentação-de-Nuvens-1/train'
images_dir = os.path.join(source_dir, 'images')
masks_dir = os.path.join(source_dir, 'masks')

# Crie as pastas 'images' e 'masks' se elas não existirem
os.makedirs(images_dir, exist_ok=True)
os.makedirs(masks_dir, exist_ok=True)

# Itere sobre os arquivos na pasta de origem
for filename in os.listdir(source_dir):
    # Ignore pastas para evitar mover as novas pastas 'images' e 'masks' dentro delas mesmas
    file_path = os.path.join(source_dir, filename)
    if os.path.isdir(file_path):
        continue

    # Verifique se o arquivo é uma máscara
    if '_mask' in filename:
        # Mova as máscaras para a pasta 'masks'
        shutil.move(file_path, os.path.join(masks_dir, filename))
    else:
        # Presume que todos os outros arquivos são imagens
        shutil.move(file_path, os.path.join(images_dir, filename))

print('Arquivos organizados nas pastas "images/" e "masks/".')


#Treinar

In [ ]:
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class SegmentationDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.images = os.listdir(images_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_name = self.images[idx]
        image_path = os.path.join(self.images_dir, image_name)

        # Substituir a extensão da imagem para encontrar a máscara correspondente
        mask_name = image_name.replace('.jpg', '_mask.png')  # ou .png, dependendo do seu formato
        mask_path = os.path.join(self.masks_dir, mask_name)

        # Carregar a imagem e a máscara
        image = Image.open(image_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # Convertido para escala de cinza

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask


In [9]:
# Defina os diretórios das imagens e máscaras
images_dir = 'Segmentação-de-Nuvens-1/train/images'  # ajuste conforme necessário
masks_dir = 'Segmentação-de-Nuvens-1/train/masks'      # ajuste conforme necessário

# Defina as transformações para imagem e máscara
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Ajuste conforme necessário
    transforms.ToTensor()
])

# Crie o dataset e o dataloader
dataset = SegmentationDataset(images_dir, masks_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)


In [ ]:
def show_images(images, masks):
    """Função para mostrar um lote de imagens e máscaras."""
    fig, axs = plt.subplots(2, len(images), figsize=(12, 6))
    for i in range(len(images)):
        # Mostrar a imagem
        axs[0, i].imshow(images[i].permute(1, 2, 0))  # Permuta as dimensões para matplotlib
        axs[0, i].set_title('Image')
        axs[0, i].axis('off')  # Oculta os eixos

        # Mostrar a máscara
        mask = masks[i].squeeze()  # Remove a dimensão extra
        axs[1, i].imshow(mask, cmap='gray')
        axs[1, i].set_title('Mask')
        axs[1, i].axis('off')  # Oculta os eixos
    plt.tight_layout()  # Ajusta os espaços entre os subplots
    plt.show()

# Pega um lote de dados
for images, masks in dataloader:
    show_images(images, masks)
    break  # Exibe apenas o primeiro lote


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)
        self.masks = os.listdir(mask_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.masks[idx])

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # Converter a máscara para escala de cinza

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        # Adicione um print para verificar a forma das máscaras
        print(f"Image shape: {image.shape}, Mask shape: {mask.shape}")

        return image, mask

# Criando o DataLoader
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

dataset = CustomDataset(images_dir, masks_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Iterando sobre o DataLoader para verificar as formas das imagens e máscaras
for images, masks in dataloader:
    print(f"Batch of images shape: {images.shape}, Batch of masks shape: {masks.shape}")
    break  # Remova ou mantenha isso para imprimir apenas o primeiro lote


In [ ]:
import torch
import torch.nn as nn

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Adicione mais camadas de codificação conforme necessário

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, out_channels, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=1),
        )

    def forward(self, x):
        x1 = self.encoder(x)
        x2 = self.decoder(x1)
        return x2

# Inicialize o modelo
model = UNet(in_channels=3, out_channels=1)  # 1 para máscara binária


In [31]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Você pode ajustar a taxa de aprendizado


In [ ]:
from tqdm import tqdm  # Importe a biblioteca tqdm

num_epochs = 100  # Defina o número de épocas
for epoch in range(num_epochs):
    # Use tqdm para mostrar a barra de progresso
    for images, masks in tqdm(dataloader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False):
        optimizer.zero_grad()  # Zerar os gradientes
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, masks)  # Cálculo da perda
        loss.backward()  # Backward pass
        optimizer.step()  # Atualizar os pesos

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

torch.save(model.state_dict(), 'unet_model.pth')
